In [1]:
#importing all the required libraries and modules
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar
import scipy.stats
import matplotlib.dates as mdates
from google.cloud import bigquery

In [2]:
#connecting notebook to gcloud bigquery to query data.
cred_file = r"C:\Users\V0V01S7\AppData\Roaming\gcloud\legacy_credentials\venkatasaiyashaswi.v@walmart.com\adc.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cred_file
def query_bq(QUERY):
    client = bigquery.Client(project='wmt-edw-sandbox')
    query_job = client.query(QUERY) # API request
    rows = query_job.result() # Waits for query to finish
    df = rows.to_dataframe()
    return df

In [3]:
code = """select A.DRVR_USER_ID, A.MARKET_NM,
B.ACTV_START_TS, B.ACTL_ACTV_END_TS, B.ACTV_DUR_MIN_QTY, B.ACTV_DT, 
from `wmt-edw-prod.WW_GEC_VM.DRVR` A inner join `wmt-edw-prod.WW_GEC_VM.DRVR_APPLN_ACTV` B on A.DRVR_USER_ID = B.DRVR_USER_ID
where A.MARKET_NM in ('Walmart Chicago','Walmart Galveston', 'Walmart New Haven', 'Walmart Carson City', 'Walmart West Orlando','Walmart Bozeman', 'Walmart San Jose North') AND B.ACTV_NM = "ONLINE"
"""
df = query_bq(code)
df["ACTV_START_TS"] = pd.to_datetime(df["ACTV_START_TS"])
df["ACTL_ACTV_END_TS"] = pd.to_datetime(df["ACTL_ACTV_END_TS"])
#df["ACTV_START_TS"] = df["ACTV_START_TS"] + pd.Timedelta(hours=2) #converting pacific time to eastern time (orlando market)
#df["ACTL_ACTV_END_TS"] = df["ACTL_ACTV_END_TS"] + pd.Timedelta(hours=2)
df

,DRVR_USER_ID,MARKET_NM,ACTV_START_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
0,lorannelson90@gmail.com,Walmart Bozeman,2021-05-23 06:25:23.142,2021-05-23 10:00:00.000,214.600000000,2021-05-23
1,katjapicard@hotmail.com,Walmart Bozeman,2021-11-25 10:19:36.291,2021-11-25 10:20:13.238,0.620000000,2021-11-25
2,katjapicard@hotmail.com,Walmart Bozeman,2021-11-25 10:20:13.238,2021-11-25 11:30:00.000,69.780000000,2021-11-25
3,jmkieff@yahoo.com,Walmart Bozeman,2021-09-01 06:04:32.887,2021-09-01 06:34:36.960,30.070000000,2021-09-01
4,jmkieff@yahoo.com,Walmart Bozeman,2021-09-01 11:03:13.518,2021-09-01 12:17:21.904,74.130000000,2021-09-01
...,...,...,...,...,...,...
285317,primettte@gmail.com,Walmart San Jose North,2022-06-28 22:57:25.991,2022-06-25 13:00:00.000,-4917.420000000,2022-06-28
285318,sengledvasquez@gmail.com,Walmart San Jose North,2022-06-28 22:57:30.859,2022-06-25 09:00:00.000,-5157.500000000,2022-06-28
285319,jacob.krause@walmart.com,Walmart San Jose North,2022-06-28 22:57:30.973,2022-03-15 16:00:00.000,-151617.500000000,2022-06-28
285320,gameboy999.aldo@gmail.com,Walmart San Jose North,2022-06-28 22:57:26.557,2022-05-26 21:00:00.000,-47637.430000000,2022-06-28


In [4]:
df = df[df["DRVR_USER_ID"].str.contains("walmart") == False]
df = df[df["DRVR_USER_ID"].str.contains("faisal4") == False]
df

,DRVR_USER_ID,MARKET_NM,ACTV_START_TS,ACTL_ACTV_END_TS,ACTV_DUR_MIN_QTY,ACTV_DT
0,lorannelson90@gmail.com,Walmart Bozeman,2021-05-23 06:25:23.142,2021-05-23 10:00:00.000,214.600000000,2021-05-23
1,katjapicard@hotmail.com,Walmart Bozeman,2021-11-25 10:19:36.291,2021-11-25 10:20:13.238,0.620000000,2021-11-25
2,katjapicard@hotmail.com,Walmart Bozeman,2021-11-25 10:20:13.238,2021-11-25 11:30:00.000,69.780000000,2021-11-25
3,jmkieff@yahoo.com,Walmart Bozeman,2021-09-01 06:04:32.887,2021-09-01 06:34:36.960,30.070000000,2021-09-01
4,jmkieff@yahoo.com,Walmart Bozeman,2021-09-01 11:03:13.518,2021-09-01 12:17:21.904,74.130000000,2021-09-01
...,...,...,...,...,...,...
285316,hmenaverse@gmail.com,Walmart San Jose North,2022-06-28 08:04:22.198,2022-06-28 08:08:56.464,4.570000000,2022-06-28
285317,primettte@gmail.com,Walmart San Jose North,2022-06-28 22:57:25.991,2022-06-25 13:00:00.000,-4917.420000000,2022-06-28
285318,sengledvasquez@gmail.com,Walmart San Jose North,2022-06-28 22:57:30.859,2022-06-25 09:00:00.000,-5157.500000000,2022-06-28
285320,gameboy999.aldo@gmail.com,Walmart San Jose North,2022-06-28 22:57:26.557,2022-05-26 21:00:00.000,-47637.430000000,2022-06-28


In [11]:
def final_ft(df):
    df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
    time_line = pd.date_range(df["dayhour"].min(),df["dayhour"].max(),freq='H')
    time_line = time_line.to_series()
    ts = pd.DataFrame(time_line,columns = ["dayhour"])
    ts = ts.set_index("dayhour")
    ts = pd.merge(ts,df,how="left",on=["dayhour"])
    ts["ACTV_DUR_MIN_QTY"] = ts["ACTV_DUR_MIN_QTY"].fillna(0)
    ts = ts.drop(["DRVR_USER_ID","ACTV_DT"],axis=1)
    ts = ts.set_index(["dayhour"],drop=True)
    ts = ts.drop_duplicates()
    ts = ts.dropna()
    ts = ts.reset_index(drop=False)
    ts['ACTV_START_TS'] = ts["ACTV_START_TS"].apply(lambda x: x.replace(second=0, microsecond=0))
    ts['ACTL_ACTV_END_TS'] = ts["ACTL_ACTV_END_TS"].apply(lambda x: x.replace(second=0, microsecond=0))
    def create_final_data(df):
        df['minutes'] = (df['ACTL_ACTV_END_TS'] - df['ACTV_START_TS']).dt.total_seconds()/60
        df = df[df['minutes']>0].reset_index(drop=True)
        # create an IntervalIndex which we can set as the axis (needed for re-indexing).
        # subtract one minute from end_time so that the minute of the termination is excluded.
        iv_idx = pd.IntervalIndex.from_arrays(df['ACTV_START_TS'],
                                              df['ACTL_ACTV_END_TS']-pd.Timedelta(minutes=1),
                                              closed='both')
        # create a new index with the extended frequency:
        new_idx = pd.date_range(df['ACTV_START_TS'].min(), df['ACTL_ACTV_END_TS'].max(), freq='min')
        # set the new index to get the extended frequency;
        # all minutes will have the value of the whole interval
        result = df['minutes'].set_axis(iv_idx).reindex(new_idx)
        # we can now calculate the duration per hour by resampling and summing the
        # boolean representation of the duration (1/0):
        result= result.fillna(0).astype(int).astype(bool).resample('H').sum()
        result.index.name = 'start_hour'
        return result
    re = create_final_data(ts)
    re = pd.DataFrame(re)
    def create_features(df, label=None):
        """
        Creates time series features from datetime index.
        """
        df = df.copy()
        df['date'] = df.index
        df['hour'] = df['date'].dt.hour
        df['dayofweek'] = df['date'].dt.dayofweek
        df['quarter'] = df['date'].dt.quarter
        df['month'] = df['date'].dt.month
        df['year'] = df['date'].dt.year
        df['dayofyear'] = df['date'].dt.dayofyear
        df['dayofmonth'] = df['date'].dt.day
        df['weekofyear'] = df['date'].dt.isocalendar().week
    
        

        X = df[['hour','dayofweek','quarter','month','year',
               'dayofyear','dayofmonth','weekofyear']]
        if label:
            y = df[label]
            return X, y
        return X

    X, y = create_features(re, label='minutes')

    ft = pd.concat([X, y], axis=1)    
    return ft

In [6]:
dfs = {}
for ID in df['DRVR_USER_ID'].value_counts().index.tolist():
    dfs[ID] = df.loc[df['DRVR_USER_ID'] == ID]

In [7]:
len(dfs)

1604

In [17]:
dfs.keys()

dict_keys(['maruserra57@gmail.com', 'behurles@yahoo.com', 'coteclan4882@gmail.com', 'perez5300blk@gmail.com', 'canelo44@yahoo.com', 'rhantelubrico2006@yahoo.com', 'jraymondi_6@hotmail.com', 'martinvz1984@gmail.com', 'hugoalejandrocr3@gmail.com', 'fdarias15@hotmail.com', 'daytonalinda48@gmail.com', 'eliomoli@hotmail.com', 'manaviche@gmail.com', 'paully1530@gmail.com', 'jkirk427@yahoo.com', 'javier8a2011@gmail.com', 's.levarity@yahoo.com', 'danaly0903@gmail.com', 'bisitenny@gmail.com', 'twgegem@gmail.com', 'rosacarolinasimozarealza@gmail.com', 'ricardobelloso20@hotmail.com', 'jessicaarcila88@gmail.com', 'derekgallegos.tm@gmail.com', 'solyaname@gmail.com', 'cmorales615@icloud.com', 'nanpgh2015@gmail.com', 'lorensmitter@gmail.com', 'alexvelasquez_05@hotmail.com', 'rottibk12@yahoo.com', 'josechanga@hotmail.com', 'lln_cook@yahoo.com', 'tkaym0309@gmail.com', 'jparrauber38@gmail.com', 'daihelly20@gmail.com', 'wmorales0275@gmail.com', 'jimenezbazan1995@gmail.com', 'samisultani65@gmail.com', 'r.

In [ ]:
dfs_set = {}
for i in dfs.keys():
    dfs_set[i] = pd.DataFrame(final_ft(dfs[i]))
    dfs_set[i]["ID"] = i
    dfs_set[i]["location"] = dfs_set[i]["ID"].apply(lambda x: df.loc[df["DRVR_USER_ID"] == x].MARKET_NM.iloc[0])

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_14724\1320972176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_14724\1320972176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_14724\1320972176.py:2: SettingWithCopyWa

C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_14724\1320972176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))
C:\Users\V0V01S7\AppData\Local\Temp\ipykernel_14724\1320972176.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dayhour'] = df["ACTV_START_TS"].apply(lambda x: x.replace(minute=0, second=0, microsecond=0))


In [27]:
dff = pd.concat([dfs_set[i] for i in dfs_set.keys()], ignore_index=False)
dff.to_csv("Desktop/dffall.csv")

In [86]:
df["ID"] = df["DRVR_USER_ID"]
dff.head()

,hour,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear,minutes,ID,slot,weekend
start_hour,,,,,,,,,,,,
2021-05-18 04:00:00,4,1,2,5,2021,138,18,20,59,maruserra57@gmail.com,None,weekday
2021-05-18 05:00:00,5,1,2,5,2021,138,18,20,60,maruserra57@gmail.com,None,weekday
2021-05-18 06:00:00,6,1,2,5,2021,138,18,20,60,maruserra57@gmail.com,morning,weekday
2021-05-18 07:00:00,7,1,2,5,2021,138,18,20,60,maruserra57@gmail.com,morning,weekday
2021-05-18 08:00:00,8,1,2,5,2021,138,18,20,60,maruserra57@gmail.com,morning,weekday


In [82]:
#dff.reset_index(drop=False)
sdff = dff[:100000]

In [85]:
sdff.merge(df[["ID","MARKET_NM"]], on='ID', how = "inner")


MemoryError: Unable to allocate 19.7 GiB for an array with shape (8, 331270696) and data type int64

In [68]:
dff["location"] = dff["ID"].apply(lambda x: df.loc[df["DRVR_USER_ID"] == x].MARKET_NM.iloc[0])

KeyboardInterrupt: 

In [30]:
def slot_calc(hour):
    
    if hour in range(6,12):
        return "morning"
    elif hour in range(12,17):
        return "afternoon"
    elif hour in range(17,22):
        return "evening"
    
def weekend_calc(dayofweek):
    if dayofweek in range(0,5):
        return "weekday"
    elif dayofweek in range(5,7):
        return "weekend"

In [31]:
dff["slot"] = dff["hour"].apply(slot_calc)
dff["weekend"] = dff["dayofweek"].apply(weekend_calc)

In [32]:
wdff = pd.DataFrame(dff.groupby(by = ["ID","weekend"])["minutes"].mean())
sdff = pd.DataFrame(dff.groupby(by = ["ID","slot"])["minutes"].mean())

In [34]:
ww = wdff.loc[wdff.groupby('ID')['minutes'].idxmax()]
sod = sdff.loc[sdff.groupby('ID')['minutes'].idxmax()]

In [38]:
sod = sod.reset_index(level = ["slot"])
ww = ww.reset_index(level=['weekend'])

In [56]:
cdf = pd.merge(ww,sod,on = "ID")
cdf["driver"] = cdf.index
cdf["location"] = cdf["driver"].apply(lambda x: df.loc[df["DRVR_USER_ID"] == x].MARKET_NM.iloc[0])

In [57]:
cdf.head()

,weekend,minutes_x,slot,minutes_y,driver,location
ID,,,,,,
1.ms2.r.3s@gmail.com,weekday,4.929553,evening,8.537037,1.ms2.r.3s@gmail.com,Walmart San Jose North
16gonzalezveronica@gmail.com,weekend,5.591581,morning,12.660305,16gonzalezveronica@gmail.com,Walmart West Orlando
1985chicitybears@gmail.com,weekday,0.886268,morning,1.848193,1985chicitybears@gmail.com,Walmart Chicago
3va.bickmore@gmail.com,weekend,7.837960,afternoon,16.206226,3va.bickmore@gmail.com,Walmart Carson City
41jennings@gmail.com,weekday,0.194793,morning,0.436439,41jennings@gmail.com,Walmart Shelby


In [60]:
pd.DataFrame(cdf.groupby(by = ["location","driver","slot"])["minutes_y"].mean())

minutes_y
location             driver                     slot                
Walmart Carson City  3va.bickmore@gmail.com     afternoon  16.206226
                     77perryj2625@gmail.com     morning     3.464799
                     afoxroxmysox@gmail.com     morning     3.884078
                     ajayeckel@hotmail.com      morning     3.377815
                     aliasnwn@charter.net       afternoon   5.714286
...                                                              ...
Walmart West Orlando yulijimenez.ygjt@gmail.com morning     3.984381
                     yurimorales2704@gmail.com  morning     0.386266
                     zeimarsanchez@gmail.com    morning     5.548969
                     zerquera34@gmail.com       morning     0.919565
                     zudleals@gmail.com         morning    43.976190

[1489 rows x 1 columns]

In [61]:
pd.DataFrame(cdf.groupby(by = ["location","driver","weekend"])["minutes_x"].mean())

minutes_x
location             driver                     weekend           
Walmart Carson City  3va.bickmore@gmail.com     weekend   7.837960
                     77perryj2625@gmail.com     weekend   3.675505
                     afoxroxmysox@gmail.com     weekend   2.689951
                     ajayeckel@hotmail.com      weekday   2.000196
                     aliasnwn@charter.net       weekend   8.644444
...                                                            ...
Walmart West Orlando yulijimenez.ygjt@gmail.com weekday   2.421139
                     yurimorales2704@gmail.com  weekday   0.196415
                     zeimarsanchez@gmail.com    weekday   2.280875
                     zerquera34@gmail.com       weekday   0.489796
                     zudleals@gmail.com         weekday  24.047619

[1489 rows x 1 columns]